In [2]:
import json
import os
from unskript import nbparams
from unskript.secrets import ENV_MODE, ENV_MODE_AWS
from unskript.fwk.workflow import Task, Workflow

env = {"ENV_MODE":"ENV_MODE_AWS","TENANT_ID":"982dba5f-d9df-48ae-a5bf-ec1fc94d4882","PROXY_ID":"1499f27c-6406-4fbd-bd1b-c6f92800018f","TENANT_URL":"https://tenant-staging.alpha.unskript.io","AWS_REGION":"us-west-2"}
secret_store_cfg = {"SECRET_STORE_TYPE":"SECRET_STORE_TYPE_AWS","AWS_SECRET_PREFIX":"staging","AWS_REGION":"us-west-2"}
os.environ["UNSKRIPT_REDIS_HOST"] = "redis-master.redis.svc.cluster.local"
os.environ["UNSKRIPT_TOKEN"] = "5c4a5754-0600-11ec-9a03-0242ac130003"
os.environ["UNSKRIPT_SIDECAR_URL"] = "http://sidecar.sidecar.svc.cluster.local"
os.environ["TENANT_URL"] = env["TENANT_URL"]
paramDict = {"app_label": None, "container": None, "deployment": None, "namespace": None, "pod": None}
unSkriptOutputParamDict = {}
paramDict.update(env)
paramDict.update(secret_store_cfg)
paramsJson = json.dumps(paramDict)
nbParamsObj = nbparams.NBParams(paramsJson)
app_label = nbParamsObj.get('app_label')
container = nbParamsObj.get('container')
deployment = nbParamsObj.get('deployment')
namespace = nbParamsObj.get('namespace')
pod = nbParamsObj.get('pod')


user_id = "6227fe0e8904e60069c27c5b"
w = Workflow(env, secret_store_cfg, None, global_vars=globals(), check_uuids=None)

<ul>
<li>Look for pod level information</li>
<li>Look for deployment</li>
<li>Look for app label</li>
</ul>

In [ ]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

import pprint
import json
from typing import Optional, Tuple
from pydantic import BaseModel, Field

from beartype import beartype
@beartype
def k8s_get_deployment_status_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def k8s_get_deployment_status(handle, deployment: str = "", namespace: str = "") -> Tuple:
    """k8s_get_deployment_status executes the command and give failed deployment list

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type deployment: str
        :param deployment: Deployment Name.

        :type namespace: str
        :param namespace: Kubernetes Namespace.

        :rtype: Tuple with status result and list of failed deployments.
    """
    result = []
    if handle.client_side_validation is not True:
        print(f"K8S Connector is invalid: {handle}")
        raise Exception("K8S Connector is invalid")

    status_details = ""
    if namespace and deployment:
        name_cmd = "kubectl get deployment " + deployment + " -n " + namespace + " -o json"
        exec_cmd = handle.run_native_cmd(name_cmd)
        status_op = exec_cmd.stdout
        status_details = json.loads(status_op)

    if not namespace and not deployment:
        name_cmd = "kubectl get deployments --all-namespaces -o json"
        exec_cmd = handle.run_native_cmd(name_cmd)
        status_op = exec_cmd.stdout
        status_details = json.loads(status_op)

    if namespace and not deployment:
        name_cmd = "kubectl get deployment -n " + namespace + " -o json"
        exec_cmd = handle.run_native_cmd(name_cmd)
        status_op = exec_cmd.stdout
        status_details = json.loads(status_op)

    if deployment and not namespace:
        name_cmd = "kubectl get deployment " + deployment + " -o json"
        exec_cmd = handle.run_native_cmd(name_cmd)
        status_op = exec_cmd.stdout
        status_details = json.loads(status_op)

    if status_details:
        if "items" in status_details:
            for items in status_details["items"]:
                namespace_name = items["metadata"]["namespace"]
                deployment_name = items["metadata"]["name"]
                replica_details = items["status"]["conditions"]
                for i in replica_details:
                    deployment_dict = {}
                    if ("FailedCreate" in i["reason"] and "ReplicaFailure" in i["type"] and
                        "True" in i["status"]):
                        deployment_dict["namespace"] = namespace_name
                        deployment_dict["deployment_name"] = deployment_name
                        result.append(deployment_dict)
                    if ("ProgressDeadlineExceeded" in i["reason"] and "Progressing" in i["type"] and
                        "False" in i["status"]):
                        deployment_dict["namespace"] = namespace_name
                        deployment_dict["deployment_name"] = deployment_name
                        result.append(deployment_dict)
        else:
            namespace_name = status_details["metadata"]["namespace"]
            deployment_name = status_details["metadata"]["name"]
            replica_details = status_details["status"]["conditions"]
            for i in replica_details:
                deployment_dict = {}
                if ("FailedCreate" in i["reason"] and "ReplicaFailure" in i["type"] and
                    "True" in i["status"]):
                    deployment_dict["namespace"] = namespace_name
                    deployment_dict["deployment_name"] = deployment_name
                    result.append(deployment_dict)
                if ("ProgressDeadlineExceeded" in i["reason"] and "Progressing" in i["type"] and
                    "False" in i["status"]):
                    deployment_dict["namespace"] = namespace_name
                    deployment_dict["deployment_name"] = deployment_name
                    result.append(deployment_dict)

    if len(result) != 0:
        return (False, result)
    return (True, None)


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "deployment": "deployment",
    "namespace": "namespace"
    }''')

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_get_deployment_status, lego_printer=k8s_get_deployment_status_printer, hdl=hdl, args=args)

In [ ]:
from pprint import pprint
from pydantic import BaseModel, Field
from kubernetes.client.rest import ApiException

from beartype import beartype
@beartype
def k8s_kubectl_get_logs_printer(data: str):
    if data is None:
        return

    print("Logs:")

    pprint(data)

@beartype
def k8s_kubectl_get_logs(handle, k8s_cli_string: str, pod_name: str, namespace:str) -> str:
    """k8s_kubectl_get_logs executes the given kubectl command

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type k8s_cli_string: str
        :param k8s_cli_string: kubectl logs {pod_name} -n {namespace}.

        :type pod_name: str
        :param pod_name: Pod Name.

        :type namespace: str
        :param namespace: Namespace.

        :rtype: String, Output of the command in python string format or
        Empty String in case of Error.
    """
    k8s_cli_string = k8s_cli_string.format(pod_name=pod_name, namespace=namespace)
    result = handle.run_native_cmd(k8s_cli_string)
    if result is None:
        print(
            f"Error while executing command ({k8s_cli_string}) (empty response)")
        return ""

    if result.stderr:
        raise ApiException(f"Error occurred while executing command {k8s_cli_string} {result.stderr}")

    data = result.stdout
    return data


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "k8s_cli_string": "f\\"kubectl logs {pod_name} -n {namespace}\\"",
    "namespace": "namespace",
    "pod_name": "pod"
    }''')

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_get_logs, lego_printer=k8s_kubectl_get_logs_printer, hdl=hdl, args=args)

In [ ]:
from pprint import pprint
from pydantic import BaseModel, Field
from kubernetes.client.rest import ApiException


from beartype import beartype
@beartype
def k8s_kubectl_describe_pod_printer(data: str):
    if data is None:
        return
    print("Pod Details:")
    pprint(data)


@beartype
def k8s_kubectl_describe_pod(handle, pod_name: str, k8s_cli_string: str, namespace: str) -> str:
    """k8s_kubectl_describe_pod executes the given kubectl command

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type k8s_cli_string: str
        :param k8s_cli_string: kubectl describe pod {pod_name} -n {namespace}.

        :type node_name: str
        :param node_name: Node Name.

        :type namespace: str
        :param namespace:Namespace

        :rtype: String, Output of the command in python string format or
        Empty String in case of Error.
    """
    k8s_cli_string = k8s_cli_string.format(
        pod_name=pod_name, namespace=namespace)
    result = handle.run_native_cmd(k8s_cli_string)
    if result.stderr:
        raise ApiException(
            f"Error occurred while executing command {result.stderr}")

    data = result.stdout
    return data


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "k8s_cli_string": "f\\"kubectl describe pod {pod_name} -n {namespace}\\"",
    "namespace": "namespace",
    "pod_name": "pod"
    }''')

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_describe_pod, lego_printer=k8s_kubectl_describe_pod_printer, hdl=hdl, args=args)

In [ ]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field
from kubernetes.client.rest import ApiException


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String
        in case of Error.
    """
    if handle.client_side_validation is not True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)

    if result is None:
        print(
            f"Error while executing command ({kubectl_command}) (empty response)")
        return ""

    if result.stderr:
        raise ApiException(f"Error occurred while executing command {kubectl_command} {result.stderr}")

    return result.stdout


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl get pod {pod} -n {namespace}\\""
    }''')

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

In [ ]:
from pydantic import BaseModel, Field
from kubernetes.client.rest import ApiException


from beartype import beartype
@beartype
def k8s_kubectl_show_metrics_pod_printer(data: str):
    if data is None:
        print("Error while executing command")
        return

    print (data)

@beartype
def k8s_kubectl_show_metrics_pod(
        handle,
        k8s_cli_string: str,
        pod_name:str,
        namespace:str
        ) -> str:
    """k8s_kubectl_show_metrics_node executes the given kubectl command

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type k8s_cli_string: str
        :param k8s_cli_string: kubectl top pod {pod_name} -n {namespace}.

        :type pod_name: str
        :param pod_name: Pod Name.

        :type namespace: str
        :param namespace: Namespace.

        :rtype: String, Output of the command in python string format or
        Empty String in case of Error.
    """
    k8s_cli_string = k8s_cli_string.format(pod_name=pod_name, namespace=namespace)
    result = handle.run_native_cmd(k8s_cli_string)
    if result is None:
        print(
            f"Error while executing command ({k8s_cli_string}) (empty response)")
        return ""

    if result.stderr:
        raise ApiException(
            f"Error occurred while executing command {k8s_cli_string} {result.stderr}")

    return result.stdout


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "k8s_cli_string": "f\\"kubectl top pod {pod_name} -n {namespace}\\"",
    "namespace": "namespace",
    "pod_name": "pod"
    }''')

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_show_metrics_pod, lego_printer=k8s_kubectl_show_metrics_pod_printer, hdl=hdl, args=args)

In [ ]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field
from kubernetes.client.rest import ApiException


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String
        in case of Error.
    """
    if handle.client_side_validation is not True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)

    if result is None:
        print(
            f"Error while executing command ({kubectl_command}) (empty response)")
        return ""

    if result.stderr:
        raise ApiException(f"Error occurred while executing command {kubectl_command} {result.stderr}")

    return result.stdout


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl get pods -l app={app_label} -n ${namespace}\\""
    }''')

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

In [ ]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field
from kubernetes.client.rest import ApiException


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String
        in case of Error.
    """
    if handle.client_side_validation is not True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)

    if result is None:
        print(
            f"Error while executing command ({kubectl_command}) (empty response)")
        return ""

    if result.stderr:
        raise ApiException(f"Error occurred while executing command {kubectl_command} {result.stderr}")

    return result.stdout


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl top pods -l app={app_label} -n ${namespace}\\""
    }''')

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

In [ ]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field
from kubernetes.client.rest import ApiException


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String
        in case of Error.
    """
    if handle.client_side_validation is not True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)

    if result is None:
        print(
            f"Error while executing command ({kubectl_command}) (empty response)")
        return ""

    if result.stderr:
        raise ApiException(f"Error occurred while executing command {kubectl_command} {result.stderr}")

    return result.stdout


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl logs -l app={app_label} -c {container} -n {namespace}\\""
    }''')

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

In [ ]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field
from kubernetes.client.rest import ApiException


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String
        in case of Error.
    """
    if handle.client_side_validation is not True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)

    if result is None:
        print(
            f"Error while executing command ({kubectl_command}) (empty response)")
        return ""

    if result.stderr:
        raise ApiException(f"Error occurred while executing command {kubectl_command} {result.stderr}")

    return result.stdout


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl describe pods -l app={app_label} -n {namespace}\\""
    }''')

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

In [ ]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field
from kubernetes.client.rest import ApiException


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String
        in case of Error.
    """
    if handle.client_side_validation is not True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)

    if result is None:
        print(
            f"Error while executing command ({kubectl_command}) (empty response)")
        return ""

    if result.stderr:
        raise ApiException(f"Error occurred while executing command {kubectl_command} {result.stderr}")

    return result.stdout


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl get events --field-selector involvedObject.name={pod} -n {namespace}\\""
    }''')

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

<p>Increase resource allocation: If the root cause is identified as insufficient resources, increase the allocation of CPU, memory, or other resources to the Kubernetes deployment.Increase resource allocation: If the root cause is identified as insufficient resources, increase the allocation of CPU, memory, or other resources to the Kubernetes deployment.</p>

In [8]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field
from kubernetes.client.rest import ApiException


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String
        in case of Error.
    """
    if handle.client_side_validation is not True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)

    if result is None:
        print(
            f"Error while executing command ({kubectl_command}) (empty response)")
        return ""

    if result.stderr:
        raise ApiException(f"Error occurred while executing command {kubectl_command} {result.stderr}")

    return result.stdout


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl set resources deployment {deployment} -n {namespace} --limits=cpu={new_cpu_limit},memory={new_memory_limit}\\""
    }''')

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

In [ ]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field
from kubernetes.client.rest import ApiException


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String
        in case of Error.
    """
    if handle.client_side_validation is not True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)

    if result is None:
        print(
            f"Error while executing command ({kubectl_command}) (empty response)")
        return ""

    if result.stderr:
        raise ApiException(f"Error occurred while executing command {kubectl_command} {result.stderr}")

    return result.stdout


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "kubectl_command": "f\\"kubectl scale deployment -n $namespace $deployment --replicas=$replicas\\"\n"
    }''')

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)